# Pré-processamento
Este notebook é responsável por pré-processar os dados realizando as seguintes tarefas:
- Tratamento de dados ausentes
- Definição de tipos
- Normalização e discretização
- Limpeza de dados (univariado, bivariado e multivariado)

Este notebook usa o 'clean_vehicles.csv' como dataset, que já não tem mais as colunas 'county', 'region_url' e 'image_url'.

In [13]:
from Utils import *
from tqdm import tqdm

DATASET = "../datasets/clean_vehicles_2.csv"
df = pd.read_csv(DATASET)

## Definição de tipos
Olhando para a tabela que descreve o dataset em '1-initial-analysis.ipynb' e pela análise feita no notebook, os dados foram definidos nos formatos abaixo.

### Colunas categóricos
- type
- region
- transmission
- manufacturer
- model
- condition
- cylinders
- fuel
- title_status
- drive
- size (removido)
- paint_color
- state

### Colunas do tipo inteiro
- id (removida)
- price
- year
- odometer

### Coluna do tipo float
- lat
- long

### Colunas do tipo object
- url
- description
- vin (removida)

## Tratamento de dados ausentes ou nulos
Antes de podermos definir os tipos dos dados vamos primeiro lidar com os dados nulos ou ausentes.

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435849 entries, 0 to 435848
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            435849 non-null  int64  
 1   url           435849 non-null  object 
 2   region        435849 non-null  object 
 3   price         435849 non-null  int64  
 4   year          434732 non-null  float64
 5   manufacturer  415102 non-null  object 
 6   model         429650 non-null  object 
 7   condition     249043 non-null  object 
 8   cylinders     269465 non-null  object 
 9   fuel          432858 non-null  object 
 10  odometer      360701 non-null  float64
 11  title_status  434043 non-null  object 
 12  transmission  433703 non-null  object 
 13  vin           239197 non-null  object 
 14  drive         313838 non-null  object 
 15  size          139888 non-null  object 
 16  type          318741 non-null  object 
 17  paint_color   300602 non-null  object 
 18  desc

In [21]:
# Verifica quais dados são nulos.
df.isnull().sum()

id                   0
url                  0
region               0
price                0
year              1117
manufacturer     20747
model             6199
condition       186806
cylinders       166384
fuel              2991
odometer         75148
title_status      1806
transmission      2146
vin             196652
drive           122011
size            295961
type            117108
paint_color     135247
description         27
state                0
lat               8235
long              8235
dtype: int64

Foi notado que as colunas 'vin' (código único do véiculo, não será útil) e 'size' ('type' já fornece a mesma informação) apresentam muitos dados nulos, mais de 50% do tamanho do dataset. Elas serão removidas. A coluna 'id' também será removida pois não será útil, é apenas um identificador da oferta no craig list.

In [26]:
#df.drop(columns=['vin', 'size', 'id'], inplace=True)
df.columns

Index(['url', 'region', 'price', 'year', 'manufacturer', 'model', 'condition',
       'cylinders', 'fuel', 'odometer', 'title_status', 'transmission',
       'drive', 'type', 'paint_color', 'description', 'state', 'lat', 'long'],
      dtype='object')

### Tratamento de dados ausentes em Dados categóricos
Para os dados categóricos iremos criar uma nova categoria 'undefined' para substituir os dados ausentes.

In [27]:
categoric_types = ['type', 'region', 'transmission', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'title_status', 'drive', 'paint_color', 'state']

for column in categoric_types:
    df[column] = df[column].fillna('undefined')

In [28]:
# confirmando que não há mais amostras com dados categoricos nulos no novo dataframe
df.isnull().sum()

url                 0
region              0
price               0
year             1117
manufacturer        0
model               0
condition           0
cylinders           0
fuel                0
odometer        75148
title_status        0
transmission        0
drive               0
type                0
paint_color         0
description        27
state               0
lat              8235
long             8235
dtype: int64

### Tratamento de dados ausentes em Dados do tipo inteiro
Para os dados do tipo inteiro foi verificado que existe dados nulos apenas na coluna 'year', poucas amostras possui valor nulo na coluna 'year', representando 0,2% do dataset. Como o ano é um fator importante para a compra e venda de carros, queremos manter essas amostras.

Foi feita uma análise e notou-se que nesses casos o ano é na maioria das vezes os 4 primeiros dígitos da descrição seguidos da marca e modelo do carro. Usamos a função 'get_year_from_description' para realizar esse processo, a definição da função está no Utils.py.

In [29]:
# mostra os 20 primeiros caracteres da descrição das linhas que possuem a coluna ano nula
for elem in df[df['year'].isnull().values]['description']:
    try:
        print(elem[0:40])
    except:
        print(elem)

2019 *Chrysler* *Pacifica* Touring Plus 
2019 *Jeep* *Grand Cherokee* Laredo SUV 
2019 *Chevrolet* *Tahoe* 2WD 4dr LT SUV 
2018 *Toyota* *Tacoma* TACOMA SR5 V6 DOU
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2018 *Subaru* *Forester* 2.5i CVT SUV - 
2018 *Volkswagen* *Jetta* 1.4T S Automat
2018 *Cadillac* *CTS-V Sedan* 4dr Sedan 
2019 *Ford* *F-150* Lariat 4x4 4dr Super
2018 *Honda* *Accord Sedan* Sport 2.0T 1
2019 *Kia* *Optima* LX Automatic Sedan -
2020 *Chevrolet* *5500XD LCF Diesel* 550
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2019 *Chevrolet* *Corvette* 2dr Grand Sp
2018 *Chevrolet* *Tahoe* LT-4X4-22S-NEW 
2018 *Honda* *Pilot* EX-L 2WD SUV - $28,
2018 *Acura* *TLX*  Sedan - $29,975Call 
2019 *Chevrolet* *Silverado 1500 LD* LT 
2019 *Mitsubishi* *Mirage* ES CVT ONLY 3
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2018 *Subaru* *Legacy* 2.5i Premium Seda
2018 *Hyundai* *Accent* SE Sedan Automat
2018 *Toyota* *Tacoma* TACOMA SR5 V6 DOU
2019 *Ford* *Super Duty F-350 DRW Cab-Ch
2019 *BMW* *X7* 

In [30]:
get_year_from_description(df)

Ainda existem dados de ano nulos (28 linhas), analisando podemos perceber que muitas das descrições são nulas ou não estão no formato esperado e nesses casos muitos outros atributos estão undefined. Iremos remover os que tiverem a coluna 'description' nula.

In [31]:
df[df['year'].isnull()]

,url,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,description,state,lat,long
15321,https://chattanooga.craigslist.org/cto/d/ten-m...,chattanooga,2700,NaN,undefined,ep liberty,excellent,6 cylinders,gas,208000.0,rebuilt,automatic,rwd,SUV,grey,03 jeep liberty 2 wheel drive v6 runs good col...,tn,35.6279,-84.7542
28526,https://wausau.craigslist.org/cto/d/rosholt-20...,wausau,4300,NaN,undefined,undefined,undefined,undefined,undefined,NaN,undefined,undefined,undefined,undefined,undefined,NaN,wi,NaN,NaN
40969,https://worcester.craigslist.org/cto/d/worcest...,worcester / central MA,4500,NaN,undefined,undefined,undefined,undefined,undefined,NaN,undefined,undefined,undefined,undefined,undefined,NaN,ma,NaN,NaN
74356,https://losangeles.craigslist.org/lgb/cto/d/la...,los angeles,2600,NaN,undefined,undefined,undefined,undefined,undefined,NaN,undefined,undefined,undefined,undefined,undefined,NaN,ca,NaN,NaN
74400,https://losangeles.craigslist.org/lac/cto/d/la...,los angeles,2600,NaN,undefined,undefined,undefined,undefined,undefined,NaN,undefined,undefined,undefined,undefined,undefined,NaN,ca,NaN,NaN
80723,https://honolulu.craigslist.org/oah/cto/d/waip...,hawaii,5200,NaN,undefined,undefined,undefined,undefined,undefined,NaN,undefined,undefined,undefined,undefined,undefined,NaN,hi,NaN,NaN
87185,https://losangeles.craigslist.org/lac/cto/d/lo...,los angeles,6000,NaN,undefined,undefined,undefined,undefined,undefined,NaN,undefined,undefined,undefined,undefined,undefined,NaN,ca,NaN,NaN
87528,https://greenville.craigslist.org/cto/d/lauren...,greenville / upstate,2000,NaN,undefined,undefined,undefined,undefined,undefined,NaN,undefined,undefined,undefined,undefined,undefined,NaN,sc,NaN,NaN
87633,https://boise.craigslist.org/cto/d/riggins-200...,boise,8200,NaN,undefined,undefined,undefined,undefined,undefined,NaN,undefined,undefined,undefined,undefined,undefined,NaN,id,NaN,NaN
89520,https://greenville.craigslist.org/cto/d/greenv...,greenville / upstate,8000,NaN,undefined,undefined,undefined,undefined,undefined,NaN,undefined,undefined,undefined,undefined,undefined,NaN,sc,NaN,NaN


In [32]:
df.dropna(subset=['description'], inplace=True)
df.isnull().sum()

url                 0
region              0
price               0
year                4
manufacturer        0
model               0
condition           0
cylinders           0
fuel                0
odometer        75124
title_status        0
transmission        0
drive               0
type                0
paint_color         0
description         0
state               0
lat              8211
long             8211
dtype: int64

#### Sobram quatro linhas em que o ano é nulo:

In [33]:
df[df['year'].isnull()]

,url,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,description,state,lat,long
15321,https://chattanooga.craigslist.org/cto/d/ten-m...,chattanooga,2700,NaN,undefined,ep liberty,excellent,6 cylinders,gas,208000.0,rebuilt,automatic,rwd,SUV,grey,03 jeep liberty 2 wheel drive v6 runs good col...,tn,35.6279,-84.7542
170030,https://sandiego.craigslist.org/nsd/cto/d/esco...,san diego,5000,NaN,undefined,yota 4x4,excellent,6 cylinders,gas,NaN,clean,automatic,4wd,pickup,white,"92 toyota 4x4 king cab automatic, v6, JBL blu...",ca,33.1039,-117.0770
170856,https://sandiego.craigslist.org/ssd/ctd/d/san-...,san diego,1000,NaN,undefined,undefined,undefined,undefined,gas,0.0,clean,automatic,undefined,undefined,undefined,Auto Auction of San Diego address: 5801 Fai...,ca,32.7808,-117.1010
333060,https://lakeland.craigslist.org/cto/d/babson-p...,lakeland,3800,NaN,undefined,oor,good,8 cylinders,gas,200000.0,clean,automatic,4wd,truck,white,Nice truck 4wD. Everything works new brakes 80...,fl,27.7618,-81.5749


In [34]:
for elem in df[df['year'].isnull()]['description']:
    print(elem)
    print('\n')

03 jeep liberty 2 wheel drive v6 runs good cold air and hot heat all windows work real nice on the inside and out it does have a reconstructed title 423twofivetwosevenzeroninesix


92 toyota 4x4 king cab automatic, v6, JBL  blue tooth stereo, clean title, 3rd owner, no accident, runs great


Auto Auction of San Diego address:    5801 Fairmount Ave Mission Valley CA, 92120phone:      ☎ (619) 430-4345 ext 32text:       Text 32 to (619) 430-4345 for more details  link:       http://inventory.autoauctionofsandiego.com/v/39002970/2/21825851/32  contact:    Auto Auction of San Diego      PUBLIC AUCTION OPENING BID Notes from the DealerMany other cars will be auctioned off this Saturday. All cars are smogged and have clean titles. Visit www.AutoAuctionOfSanDiego.com to see more cars. Vehicle InfoStock #: 25334VIN: LOHMT0023HCN00488Condition: UsedDescription Big PUBLIC Auto Auction this Saturday 11am in Mission Valley! The OPENING BID for this vehicle is $1000. The Auto Auction is OPEN TO THE 

Pela descrição so consegue-se informação do ano para linha com indice 170030 e indice 15321, o toyota de ano 92 e o jeep 2003, respectivamente.

In [35]:
df.at[170030, 'year'] = 1992
df.at[15321, 'year'] = 2003

As outras 2 linhas vamos descartar

In [41]:
df.dropna(subset=['year'], inplace=True)
df.isnull().sum()

url                 0
region              0
price               0
year                0
manufacturer        0
model               0
condition           0
cylinders           0
fuel                0
odometer        75124
title_status        0
transmission        0
drive               0
type                0
paint_color         0
description         0
state               0
lat              8211
long             8211
dtype: int64

O ano foi inferido como float64 pelo pandas, verificamos se há ou não casas decimais no com a função 'get_decimal' definida no script Utils.py para cada linha do dataframe:

In [196]:
df['year'].dtype

dtype('float64')

Como a serie retornada é vazia, todos os anos são números inteiros.

In [203]:
# testa se existe algum ano do tipo 2019.2 ou 2019.1, que possa ser entendido como float
test = df['year'].apply(get_decimal)
test = test[test > 0]
test

Series([], Name: year, dtype: float64)

#### Coluna 'odometer'
Vamos tentar pegar informações sobre o odômetro na descrição da oferta. 

In [14]:
# quantidade de nulos na coluna odometer
df['odometer'].isnull().sum()

75124

miles_df contém todas entradas com coluna odometer nula e que a descrição contém a alguma informação sobre as milhas do carro

In [15]:
miles_df = df[df['odometer'].isnull() & df['description'].str.contains('[Mm]ile[age]*')]
miles_df

,url,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,description,state,lat,long
15,https://mohave.craigslist.org/ctd/d/lake-havas...,mohave county,10988,2011.0,chevrolet,suburban,undefined,undefined,gas,NaN,clean,automatic,undefined,undefined,undefined,"2011 Chevy Suburban LS with 184,626 miles 5.3L...",az,34.5575,-114.3310
16,https://mohave.craigslist.org/ctd/d/lake-havas...,mohave county,16988,2012.0,nissan,titan pro-4x,undefined,undefined,gas,NaN,clean,automatic,undefined,undefined,undefined,"2012 Nissan Titan PRO-4X with 93,360 miles 5.6...",az,34.5575,-114.3310
22,https://maine.craigslist.org/cto/d/waterville-...,maine,2200,1999.0,gmc,undefined,good,8 cylinders,gas,NaN,clean,automatic,4wd,SUV,black,STICKERED TILL NOVEMBER!!! Runs and drives!!! ...,me,44.5543,-69.6178
53,https://mohave.craigslist.org/ctd/d/lake-havas...,mohave county,3388,2002.0,hyundai,sonata gls,undefined,undefined,gas,NaN,clean,automatic,undefined,undefined,undefined,"2002 Hyundai Sonata GLS with 82,038 miles 2.7L...",az,34.5575,-114.3310
58,https://milwaukee.craigslist.org/ctd/d/ironwoo...,milwaukee,6999,2007.0,dodge,nitro,undefined,undefined,gas,NaN,clean,automatic,undefined,undefined,undefined,"2007 Dodge Nitro-4X4-Leather- 116,000 Miles- L...",wi,46.4638,-90.1588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435770,https://oregoncoast.craigslist.org/ctd/d/eugen...,oregon coast,10990,2014.0,ford,focus titanium,excellent,undefined,gas,NaN,clean,automatic,undefined,undefined,undefined,We are open to serve our community! With the...,or,44.0451,-123.0440
435774,https://mohave.craigslist.org/cto/d/lake-havas...,mohave county,7850,2011.0,chevrolet,cruze,undefined,undefined,gas,NaN,clean,automatic,undefined,undefined,undefined,Extremely Nice 2011 Chevy Cruze LTZ RS model. ...,az,34.5575,-114.3310
435779,https://milwaukee.craigslist.org/ctd/d/ironwoo...,milwaukee,6999,2008.0,dodge,grand caravan,undefined,undefined,gas,NaN,clean,automatic,undefined,undefined,undefined,"V6 Stow & Go Seating 6 Passenger Only 93,000 M...",wi,46.4638,-90.1588
435800,https://portland.craigslist.org/clc/ctd/d/glad...,portland,9995,2007.0,toyota,4runner sr5 sr5 4dr suv,undefined,undefined,gas,NaN,clean,automatic,4wd,SUV,silver,2007 Toyota 4Runner SR5 SR5 4dr SUV SUV2007 TO...,or,45.4096,-122.5690


In [16]:
i = 0
erros = 0
for idx, row in tqdm(miles_df.iterrows()):
    description = df.loc[idx]['description']
    mile = re.findall(r'[Mm][Ii][Ll][Ee][Aa][Gg][Ee]:\s*\d+[Kk,.]*\d*|\d+[Kk,.]*\d*\s*[Mm][Ii][Ll][Ee][Ss]', description)

    if(mile != None and len(mile) >= 1):
        i += 1
        aux = 0
        mile = [re.findall(r'\d+[Kk,.]*\d*', item) for item in mile]
        mile = [format_mile_info(item[0]) for item in mile]
        try:
            for item in mile:
                i_item = int(item)
                if 1000 < i_item < 1000000 and i_item > aux:
                    aux = i_item
            df.at[idx, 'odometer'] = aux
        except:
            erros += 1
        #print(mile)

print(i, erros, sep='\n')

39571it [00:20, 1941.96it/s]

29213
66


In [23]:
df.isnull().sum()

url                 0
region              0
price               0
year                0
manufacturer        0
model               0
condition           0
cylinders           0
fuel                0
odometer        45977
title_status        0
transmission        0
drive               0
type                0
paint_color         0
description         0
state               0
lat              8211
long             8211
dtype: int64

In [31]:
teste = df[df['odometer'].notnull()]['odometer'].apply(get_decimal)
teste = teste[teste > 0]
teste

Series([], Name: odometer, dtype: float64)

In [33]:
df.to_csv('../datasets/clean_vehicles_3.csv', index=False)

### Tratamento de dados ausentes em Dados do tipo object
Como retiramos a coluna 'vin' do dataset, não há mais colunas de tipo object com valores nulos.

In [45]:
df[['url', 'description']].isnull().sum()

url            0
description    0
dtype: int64

### Tratamento de dados ausentes em Dados do tipo Float
#### Latitude e Longitude

In [9]:

df['mileage'] = -1

39571it [00:21, 1827.21it/s]

29213
66


In [7]:
df

,url,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,description,state,lat,long,mileage
0,https://mohave.craigslist.org/ctd/d/lake-havas...,mohave county,3495,2012.0,jeep,patriot,like new,4 cylinders,gas,NaN,clean,automatic,undefined,undefined,silver,"THIS 2012 JEEP PATRIOT IS A 4CYL. AC, STEREO, ...",az,34.4554,-114.2690,132000
1,https://oregoncoast.craigslist.org/cto/d/warre...,oregon coast,13750,2014.0,bmw,328i m-sport,good,undefined,gas,76237.0,clean,automatic,rwd,sedan,grey,Selling my 2014 BMW 328i with the following be...,or,46.1837,-123.8240,-1
2,https://greenville.craigslist.org/cto/d/sparta...,greenville / upstate,2300,2001.0,dodge,caravan,excellent,6 cylinders,gas,199000.0,clean,automatic,undefined,undefined,undefined,"01 DODGE CARAVAN,3.3 ENGINE,AUT TRANS,199000 M...",sc,34.9352,-81.9654,199000
3,https://mohave.craigslist.org/cto/d/lake-havas...,mohave county,9000,2004.0,chevrolet,colorado ls,excellent,5 cylinders,gas,54000.0,clean,automatic,rwd,pickup,red,"2004 Chevy Colorado LS, ONLY 54000 ORIGINAL MI...",az,34.4783,-114.2710,-1
4,https://maine.craigslist.org/ctd/d/searsport-t...,maine,0,2021.0,undefined,Honda-Nissan-Kia-Ford-Hyundai-VW,undefined,undefined,other,NaN,clean,other,undefined,undefined,undefined,CALL: 207.548.6500 TEXT: 207.407.5598 **WE FI...,me,44.4699,-68.8963,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435815,https://mohave.craigslist.org/cto/d/hualapai-2...,mohave county,2500,2005.0,ford,f150,fair,undefined,gas,282866.0,clean,automatic,undefined,truck,white,"2005 F150 XLT 5.4 TRITON NEEDS ENGINE, 2WD. A...",az,35.2414,-113.9890,-1
435816,https://rapidcity.craigslist.org/cto/d/rapid-c...,rapid city / west SD,2700,2002.0,toyota,camry,good,6 cylinders,gas,194000.0,clean,automatic,fwd,undefined,blue,"2002 Toyota Camry. 4-door, navy blue, good ti...",sd,44.0036,-103.3590,194000
435817,https://oregoncoast.craigslist.org/cto/d/corne...,oregon coast,2450,2001.0,ford,focus,good,4 cylinders,gas,130484.0,clean,automatic,rwd,other,black,"I am selling my ford focus 2001 SE, 130 thousa...",or,45.5272,-123.0950,-1
435818,https://oregoncoast.craigslist.org/ctd/d/portl...,oregon coast,8995,2013.0,mazda,mazda3,undefined,undefined,gas,93339.0,clean,automatic,fwd,sedan,undefined,Simple Easy Car BuyingNo Games No Gimmicks ...,or,45.5231,-122.5790,93339


In [7]:
n_str = '192K'
transform_k_to_thousand(n_str)

'192000'

In [30]:
n_str = '1992,0.0'
re.sub('[.,]', '', n_str)

'199200'

In [32]:
tste = [[1,2]]
tste[0][1]

2